## 导包

In [1]:
# 公榜分数0.92860690761
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import StratifiedKFold, KFold
from matplotlib.pyplot import plot, show

In [2]:
import os
import pandas as pd 
import numpy as np
import scorpyo as sp

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import time

from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from lightgbm import LGBMClassifier

In [3]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


## results
path_output_report = os.path.join(path_results, '01_原始数据探察.xlsx')

y_label = "is_risk"

## 数据读取

In [4]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

In [5]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,guojianping9672,rd,GVhZtW4i1,rqRxAjAL1RYC,firefox_78,firefox,win,win10,2022-01-18 19:10:41,内网,200,成都,2umVQwhiiwNJ,xxx.com/mail,2022-01,0
1,1,yangtao1740,sales,l3MuTMPoQ,iKPTa3su50y7,chrome_93,chrome,win,win11,2022-04-01 17:04:00,内网,200,深圳,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0
2,2,wangying9098,rd,4uHWcskWv,1baNbqxMWcCu,ie_11,ie,win,win10,2022-03-01 15:53:49,内网,200,成都,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-03,0
3,3,liguixiang3860,rd,mQh3NwtY7,C04Llg4lKl4C,edge_93,edge,win,win10,2022-02-07 19:46:25,内网,200,北京,nHrKgKdJ1Mzt,xxx.com/github,2022-02,0
4,4,guanyu9205,sales,C2QtgDKAZ,kSscjiRSz1aD,edge_93,edge,win,win10,2022-04-12 10:05:19,内网,200,成都,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0


In [6]:
# _ = sp.excel_label(df_row_train, y=y_label, excel_path=path_output_report, show=True)

In [7]:
# sp.excel_detect(df_row_train, sheet_name='02.训练集-数据描述性统计',excel_path=path_output_report)

In [8]:
# sp.excel_detect(df_row_val, sheet_name='03.测试集-数据描述性统计',excel_path=path_output_report)

## 特征工程

特征衍生

In [9]:
df = pd.concat([df_row_train, df_row_val])
# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 几点钟
df['hour'] = df['op_datetime'].dt.hour
# 周几
df['dayofweek'] = df['op_datetime'].dt.dayofweek
# 一个月的第几天
df['day'] = df['op_datetime'].dt.day
# 一年的第几月
df['month'] = df['op_datetime'].dt.month


# 用户名-认证时间
df = df.sort_values(by=['user_name', 'op_datetime']).reset_index(drop=True)
# datetime转int时间戳
df['ts'] = df['op_datetime'].values.astype(np.int64) // 10 ** 9
# 按客户进行统计这次认证和上次认证的时间差
df['ts1'] = df.groupby('user_name')['ts'].shift(1)
# 按客户进行统计这次认证和上上次认证的时间差
df['ts2'] = df.groupby('user_name')['ts'].shift(2)
# 按客户进行统计这次认证和上上上次认证的时间差
df['ts3'] = df.groupby('user_name')['ts'].shift(3)
df['ts_diff1'] = df['ts1'] - df['ts']
df['ts_diff2'] = df['ts2'] - df['ts']
df['ts_diff3'] = df['ts3'] - df['ts']

df['hour_sin'] = np.sin(df['hour']/24*2*np.pi)
df['hour_cos'] = np.cos(df['hour']/24*2*np.pi)

In [10]:
cat_f = ['user_name', 'department', 'ip_transform', 'device_num_transform', 'browser_version', 'browser',
          'os_type', 'os_version', 'ip_type', 'op_city', 'log_system_transform', 'url',]
for f in cat_f:
#     le = LabelEncoder()
#     df[f] = le.fit_transform(df[f])
    df[f+'_ts_diff_mean'] = df.groupby([f])['ts_diff1'].transform('mean')
    df[f+'_ts_diff_std'] = df.groupby([f])['ts_diff1'].transform('std')
    df[f+'_ts_diff2_mean'] = df.groupby([f])['ts_diff2'].transform('mean')
    df[f+'_ts_diff2_std'] = df.groupby([f])['ts_diff2'].transform('std')
    df[f+'_ts_diff3_mean'] = df.groupby([f])['ts_diff3'].transform('mean')
    df[f+'_ts_diff3_std'] = df.groupby([f])['ts_diff3'].transform('std')

df_train = df[df[y_label].notna()].reset_index(drop=True)
df_test = df[df[y_label].isna()].reset_index(drop=True)

In [11]:
bins = sp.woebin(df_train[cat_f + [y_label]], y_label)

train_woe = sp.woebin_ply(df_train, bins)
val_woe  = sp.woebin_ply(df_test, bins)


>>> There are 3 variables have too many unique non-numberic values, which might cause the binning process slow. Please double check the following variables: 
user_name, ip_transform, device_num_transform
>>> Continue the binning process?
1: yes 
2: no 
3: remove these variables
Selection: 3


There are 1 variables have only one binning intervals，please check the binning result. 
 (ColumnNames: ip_type)


[INFO] converting into woe values ...
[INFO] converting into woe values ...


In [12]:
def is_numeric(series):
    """
    功能: 
    1. 检查series中数据是否为数值型数据
    
    输入:
    series: Series, 列数据。
    
    输出:
    bool, false series为非数值型， true series为数值型
    
    维护记录:
    1. edited by toad
    2. altered by 黎良山 2022/01/11
    """

    return series.dtype.kind in 'ifc' 

In [13]:
feats = []
for col in train_woe.columns:
    if is_numeric(train_woe[col]) and col != y_label:
        feats.append(col)

In [14]:
df_train = train_woe
df_test = val_woe

## modeling

In [18]:
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,

    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [19]:
fold_num = 4
seeds = [2222]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label])
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label])
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.937106
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.938796
[3]	valid_0's auc: 0.93906
[4]	valid_0's auc: 0.939127
[5]	valid_0's auc: 0.937241
[6]	valid_0's auc: 0.937181
[7]	valid_0's auc: 0.938251
[8]	valid_0's auc: 0.937761
[9]	valid_0's auc: 0.937971
[10]	valid_0's auc: 0.937832
[11]	valid_0's auc: 0.938417
[12]	valid_0's auc: 0.93812
[13]	valid_0's auc: 0.938179
[14]	valid_0's auc: 0.937938
[15]	valid_0's auc: 0.937891
[16]	valid_0's auc: 0.937948
[17]	valid_0's auc: 0.937961
[18]	valid_0's auc: 0.93826
[19]	valid_0's auc: 0.938245
[20]	valid_0's auc: 0.938266
[21]	valid_0's auc: 0.938221
[22]	valid_0's auc: 0.938255
[23]	valid_0's auc: 0.938291
[24]	valid_0's auc: 0.938313
[25]	valid_0's auc: 0.938353
[26]	valid_0's auc: 0.938347
[27]	valid_0's auc: 0.938364
[28]	valid_0's auc: 0.938388
[29]	valid_0's auc: 0.938188
[30]	valid_0's auc: 0.938169
[31]	valid_0's auc: 0.938358
[32]	valid_0's auc: 0.938163
[33]	val

[182]	valid_0's auc: 0.943351
[183]	valid_0's auc: 0.943316
[184]	valid_0's auc: 0.943175
[185]	valid_0's auc: 0.943061
[186]	valid_0's auc: 0.943015
[187]	valid_0's auc: 0.943124
[188]	valid_0's auc: 0.943056
[189]	valid_0's auc: 0.942905
[190]	valid_0's auc: 0.942885
[191]	valid_0's auc: 0.942884
[192]	valid_0's auc: 0.94282
[193]	valid_0's auc: 0.942876
[194]	valid_0's auc: 0.94322
[195]	valid_0's auc: 0.943156
[196]	valid_0's auc: 0.943082
[197]	valid_0's auc: 0.943098
[198]	valid_0's auc: 0.94307
[199]	valid_0's auc: 0.943116
[200]	valid_0's auc: 0.943071
[201]	valid_0's auc: 0.943018
[202]	valid_0's auc: 0.942996
[203]	valid_0's auc: 0.942974
[204]	valid_0's auc: 0.943138
[205]	valid_0's auc: 0.943178
[206]	valid_0's auc: 0.943153
[207]	valid_0's auc: 0.943158
[208]	valid_0's auc: 0.943191
[209]	valid_0's auc: 0.943105
[210]	valid_0's auc: 0.943161
[211]	valid_0's auc: 0.942984
[212]	valid_0's auc: 0.942846
[213]	valid_0's auc: 0.942707
[214]	valid_0's auc: 0.942768
[215]	valid_0

[1]	valid_0's auc: 0.935396
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.938511
[3]	valid_0's auc: 0.938516
[4]	valid_0's auc: 0.938728
[5]	valid_0's auc: 0.940616
[6]	valid_0's auc: 0.940575
[7]	valid_0's auc: 0.937073
[8]	valid_0's auc: 0.937919
[9]	valid_0's auc: 0.938126
[10]	valid_0's auc: 0.937546
[11]	valid_0's auc: 0.936157
[12]	valid_0's auc: 0.936241
[13]	valid_0's auc: 0.936275
[14]	valid_0's auc: 0.936357
[15]	valid_0's auc: 0.936689
[16]	valid_0's auc: 0.936677
[17]	valid_0's auc: 0.936676
[18]	valid_0's auc: 0.936726
[19]	valid_0's auc: 0.936802
[20]	valid_0's auc: 0.936842
[21]	valid_0's auc: 0.937006
[22]	valid_0's auc: 0.937057
[23]	valid_0's auc: 0.937137
[24]	valid_0's auc: 0.937168
[25]	valid_0's auc: 0.937215
[26]	valid_0's auc: 0.937215
[27]	valid_0's auc: 0.937227
[28]	valid_0's auc: 0.937239
[29]	valid_0's auc: 0.937427
[30]	valid_0's auc: 0.937388
[31]	valid_0's auc: 0.937396
[32]	valid_0's auc: 0.937232
[33]	valid_0's auc:

,name,importance
19,id,250019.011065
48,http_status_code,64799.202211
0,url_ts_diff2_mean,31624.259708
73,hour_cos,28188.561453
4,ip_transform_ts_diff2_std,26553.619009
15,ts_diff3,12756.834177
3,ts_diff1,11210.796272
23,hour,8704.349482
35,day,4348.957721
54,dayofweek,3859.830378


0.9415902716811034

0.0018610166050272588